In [1]:
import pandas as pd 

movie_data = pd.read_csv('movie_metadata.csv')

In [2]:
movie_data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Unnamed: 28
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,NaN
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,NaN
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,NaN
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,NaN
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,NaN


In [3]:
# title = list(movie_data['movie_title'][movie_data['title_year'] > 2000])
title = list(movie_data['movie_title'])

In [4]:
import re

for idx, t in enumerate(title):
    title[idx] = (re.sub("[*.-/(/)/]", "", t)).strip()

title_list = title

In [5]:
from urllib import request
from bs4 import BeautifulSoup


p = re.compile('(http[s]?(://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)?)')
count=0
tweet_data = []
for _title in title_list:
    title = re.sub('[^A-Za-z0-9_]', "", _title)
    title_url = title.replace(" ", "%20")

    keyword = "https://twitter.com/search?q="+title_url+"&l=en&src=typd"
    #print(keyword)
    crawling_page = request.urlopen(keyword).read().decode('utf-8')

    soup = BeautifulSoup(crawling_page, 'html.parser')

    tweet_div = soup.findAll('div', attrs={'class':"original-tweet"})

    for div in tweet_div:        
        reply_count = div.find('span', attrs={'class':'ProfileTweet-actionCountForPresentation'})
        
        try:
            if (reply_count.text).isdigit() == True and int(reply_count.text) >= 2:

                path = div.get('data-permalink-path')

                corpus_url = "https://twitter.com/"+path
                corpus_data = request.urlopen(corpus_url).read()
                corpus_soup = BeautifulSoup(corpus_data, 'html.parser')
                permalink_data = corpus_soup.find('div', attrs={'class':'permalink', 'role':'main'})

                _original_tweet = permalink_data.find('div', attrs={'class':'js-original-tweet'})
                original_tweet = _original_tweet.find('p', attrs={'class':'tweet-text'})
        
                permalink_replies = permalink_data.find('div', attrs={'class':'replies-to'})
                _thread_tweet = permalink_replies.find('li', attrs={'class':'ThreadedConversation'})
                
                if _thread_tweet != None:
                    thread_tweet = _thread_tweet.findAll('div', attrs={'class': 'ThreadedConversation-tweet'})
    
                    search_tweet = {}
                    search_tweet['title'] = _title
    
                    tweet_conversation = []
                    tweet_conversation.append(original_tweet.text)
                
                    for tweet in thread_tweet:
                        _tweet_text = tweet.find('div', attrs={'class': 'js-tweet-text-container'})    
                        tweet_text = (_tweet_text.find('p', attrs={'class': 'tweet-text'})).text

                        tweet_conversation.append(tweet_text)
                
                    search_tweet['tweet'] = tweet_conversation
                
                    tweet_data.append(search_tweet)
                
                else:
                    pass
        
        except:
            pass
    
    count += 1
    if count % 200 == 0:
        print(count)

200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000


In [6]:
len(tweet_data)

7919

In [7]:
for idx, tweet in enumerate(tweet_data):
    __cleaned_text = [[i for i in text.split(' ') 
                       if 'http' not in i and 'twitter.com' not in i and '@' not in i] for text in tweet['tweet']]
    
    _cleaned_text = [' '.join(i) for i in __cleaned_text]
    
    cleaned_text = [' '.join(text.split('\n')) for text in _cleaned_text]
    tweet_data[idx]['tweet'] = [(re.sub("[*.-/(/)/#]", "", t)).strip().lower() for t in cleaned_text]

tweet_data

[{'title': 'Avatar',
  'tweet': ['made a ref to help people draw their fireemblemheroes',
   'here are the refs i used to make the tutorial',
   'youre a goddess thank yiu so much',
   'np, i gotchu >:3']},
 {'title': 'Avatar',
  'tweet': ['new avatar is industrial-grade trolling of trump wh retweet if u want rosie to play bannon on come on, snl just do',
   "bannon's skin hasn't looked that good in decades, i'd imagine needs a gin blossom too",
   'is too kind bannon always looks like they shoveled him out of the gutter in front of a tavern',
   'i thought i saw him sleeping in a bathroom stall at the bus station once!',
   "it's so true  if i showed up at work looking like bannon, my boss would ask if i was ok",
   'what about kcon and that birds nest on her head they all look cheap n dirty',
   'whatever do you',
   'hahahahaha there goes the tears hahaha']},
 {'title': 'Avatar',
  'tweet': ['the last one',
   'this day just keeps getting better and better',
   'surviving without to

In [8]:
tweet_data

[{'title': 'Avatar',
  'tweet': ['made a ref to help people draw their fireemblemheroes',
   'here are the refs i used to make the tutorial',
   'youre a goddess thank yiu so much',
   'np, i gotchu >:3']},
 {'title': 'Avatar',
  'tweet': ['new avatar is industrial-grade trolling of trump wh retweet if u want rosie to play bannon on come on, snl just do',
   "bannon's skin hasn't looked that good in decades, i'd imagine needs a gin blossom too",
   'is too kind bannon always looks like they shoveled him out of the gutter in front of a tavern',
   'i thought i saw him sleeping in a bathroom stall at the bus station once!',
   "it's so true  if i showed up at work looking like bannon, my boss would ask if i was ok",
   'what about kcon and that birds nest on her head they all look cheap n dirty',
   'whatever do you',
   'hahahahaha there goes the tears hahaha']},
 {'title': 'Avatar',
  'tweet': ['the last one',
   'this day just keeps getting better and better',
   'surviving without to

In [9]:
import pickle
 
with open('movie_tweet_2.pkl', 'wb') as f:
    pickle.dump(tweet_data, f)